# Polymorphism and Special Methods

Notes from [Deep Dive 4](https://www.udemy.com/course/python-3-deep-dive-part-4/) section 4. Topics covered:

1\. [Introduction](#introduction)

2\. [\_\_str\_\_ and \_\_repr\_\_](#str&repr)

3\. [Arithmetic operators](#arithmetic-operators)

* [Addition and subtraction](#add&sub)
* [Multiplication](#mul)
* [Dot product and cross product](#dot&cross)
* [Inplace](#inplace)
* [Negation and abs](#neg&abs)

4\. [Rich Comparisons](#rich-comparisons)
* [Equal and not equal](#eq&ne)
* [Lower than and greater than](#lt&gt)

5\. [Hashing and Equality](#hashing-and-equality)

6\. [Booleans](#booleans)

7\. [Callables](#callables)

8\. [Delete method](#delete-method)

9\. [Format method](#format-method)

<hr>

<a id='introduction'></a>
## 1. Introduction

__Polymorphism__:

The ability to define a generic type of behaviour that will (potentially) behave different when applied to different types

Operators such as `+`, `-`, `*`, `/` are polymorphic because they work with different types (integer, floats, decimals, complex numbers, list, tuples, custom objects)

__Special methods__:

Once definde in custom classes provide functionality that Python can use

`__init__` - used during class instantiation

`__enter__` & `__exis__` - contex managers `with ctx() as obj:`

`__getitem__`, `__setitem__`, `__delitem__` - sequence types: `a[i]`, `a[i:j]`, `del a[i]`

`__iter__`, `__next__` - iterables and iterators `iter()`, `next()`

`__len__` - implements `len()` 

`__contains__` - implements `in`

<hr> 

<a id='str&repr'></a>
## 2. \_\_str\_\_ and \_\_repr\_\_

* both used for creating a string representation of an object

* typically `__repr__` used by developers and is called with `repr()`

* `__str__` is called with `str()` and `print()` functions

* typically `__str__` used to display purposes to end user, logging etc.

* if `__str__` not implemented, Python will look for `__repr__` instead

In [1]:
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
        
    def __repr__(self):
        print('__repr__ called')
        return f"Person(name='{self.name}, age=self.age')"
    
    def __str__(self):
        print('__str__ called')
        return self.name

In [2]:
p = Person('Python', 30)

In [3]:
p

__repr__ called


Person(name='Python, age=self.age')

In [4]:
print(p)

__str__ called
Python


In [5]:
str(p)

__str__ called


'Python'

In [6]:
repr(p)

__repr__ called


"Person(name='Python, age=self.age')"

<hr>

<a id='arithmetic-operators'></a>
## 3. Arithmetic operators

__Arithmetic operators__: <br>
`__add__` (`+`) <br>
`__sub__` (`-`) <br>
`__mul__` (`*`) <br>
`__truediv__` (`/`) <br>
`__floordiv__` (`//`) <br>
`__mod__` (`%`) <br>
`__pow__` (`**`) <br>
`__matmul__` (`@`) - new in v3.5

To indicate operation is not supported, implement method and `return NotImplemented`

<hr>

__Reflected operators__: <br>
* for `a + b` Python will call `a.__add__(b)`
* if this is `NotImplemented` AND operators are `not of the same type` python will try `b.__add__(a)` instead

`__radd__` <br>
`__rsub__` <br>
`__rmul__` <br>
`__rtruediv__` <br>
`__rfloordiv__` <br>
`__rmod__` <br>
`__rpow__` <br>

<hr>

__Inplace operators__:<br>
`__iadd__` (`+=`) <br>
`__isub__` (`-=`) <br>
`__imul__` (`*=`) <br>
`__itruediv__` (`/=`) <br>
`__ifloordiv__` (`//=`) <br>
`__imod__` (`%=`) <br>
`__ipow__` (`**=`) <br>

<hr>

__Unary operators, functions__:<br>
`__neg__` (`-a`)<br>
`__pos__` (`+a`)<br>
`__abs__` (`abs(a)`)<br>

<hr>

__Example 2__:

Implement a Vector class that supports various arithmetic operations. Assume arguments to be Real numbers.

In [7]:
from numbers import Real
from math import sqrt

class Vector:
    def __init__(self, *components):
        # validate number of components is at least one and all of them are real numbers
        if len(components) < 1:
            raise ValueError('Cannot create an empty Vector.')
        for component in components:
            if not isinstance(component, Real):
                raise ValueError(f'Vector components must all be Real numbers - {component} is invalid.')
        
        # use immutable storage for vector
        self._components = tuple(components)
    
    def __len__(self):
        return len(self._components)
    
    @property
    def components(self):
        return self._components
    
    def __repr__(self):
        # works - but unwieldy for high dimension vectors
        return f'Vector{self._components}'
    
    def validate_type_and_dimension(self, v):
        return isinstance(v, Vector) and len(v) == len(self)
    
    def __add__(self, other):
        if self.validate_type_and_dimension(other):
            print('__add__ called...')
            components = (x + y for x, y in zip(self.components, other.components))
            return Vector(*components)
        return NotImplemented
    
    def __sub__(self, other):
        if self.validate_type_and_dimension(other):
            print('__sub__ called...')
            components = (x - y for x, y in zip(self.components, other.components))
            return Vector(*components)
        return NotImplemented
    
    def __mul__(self, other):
        ''' Multiplication by a scalar value. '''
        if isinstance(other, Real):
            print('__mul__ called for scalar...')
            components = (other * x for x in self.components)
            return Vector(*components)
        if self.validate_type_and_dimension(other):
            print('__mul__ called for dot product...')
            components = (x * y for x, y in zip(self.components, other.components))
            return sum(components)
        return NotImplemented
    
    def __rmul__(self, other):
        ''' To allow scalar * Vector multiplication. '''
        print('__rmul__ called...')
        return self * other
    
    def __matmul__(self, other):
        ''' Placeholder for cross product of two vectors. 
        Not implemented as too complex for thix example.'''
        print('__matmul__ called...')
    
    def __iadd__(self, other):
        ''' Mutates Vector object with += '''
        print('__iadd__ called...')
        if self.validate_type_and_dimension(other):
            components = (x + y for x, y in zip(self.components, other.components))
            self._components = tuple(components) # mutating Vector object
            return self
        return NotImplemented
    
    def __neg__(self):
        print('__neg__ called...')
        components = (-x for x in self.components)
        return Vector(*components)
    
    def __abs__(self):
        print('__abs__ called...')
        return sqrt(sum(x ** 2 for x in self.components))

<hr>

<a id='add&sub'></a>
__Addition and subtraction__

In [8]:
v1 = Vector(1, 2)
v2 = Vector(10, 10)
v3 = Vector(1, 2, 3, 4)

In [9]:
v1

Vector(1, 2)

In [10]:
v1 + v2

__add__ called...


Vector(11, 12)

In [11]:
v2 + v1

__add__ called...


Vector(11, 12)

In [12]:
v2 - v2

__sub__ called...


Vector(0, 0)

In [13]:
try:
    print(v1 + v3)
except TypeError as ex:
    print(ex)

unsupported operand type(s) for +: 'Vector' and 'Vector'


In [14]:
try:
    print(v1 + 100)
except TypeError as ex:
    print(ex)

unsupported operand type(s) for +: 'Vector' and 'int'


<hr>

<a id='mul'></a>
__Multiplication__

In [15]:
v1 * 10

__mul__ called for scalar...


Vector(10, 20)

In [16]:
try:
    10 * v1
except TypeError as ex:
    print('TypeError:', ex)

__rmul__ called...
__mul__ called for scalar...


What happened above:
1. Python tried to call `mul` operation on the `int` object but integers don't support custom class Vector type.
2. Python tried using `Vector` class but not the `__mul__` since that is called once Vector is the __left__ operand. 
3. Python used `__rmul__` as `Vector` is the __right__ operand, which calls Vector * scalar.

<hr>

<a id='dot&cross'></a>
__Dot product__ and __cross product__ of two vectors

In [17]:
v1 * v2

__mul__ called for dot product...


30

In [18]:
v1 @ v2

__matmul__ called...


<hr>

<a id='inplace'></a>
__Inplace__ operators:

* Mutates Vector object, which means the same object `id` remains

In [19]:
v1, id(v1)

(Vector(1, 2), 1659417711088)

In [20]:
v1 += v2

__iadd__ called...


In [21]:
v1, id(v1)

(Vector(11, 12), 1659417711088)

<hr>

<a id='neg&abs'></a>
__Negation__ and __abs__:

In [22]:
v1, v2

(Vector(11, 12), Vector(10, 10))

In [23]:
-v1

__neg__ called...


Vector(-11, -12)

In [24]:
v1 + -v2

__neg__ called...
__add__ called...


Vector(1, 2)

In [25]:
abs(v1)

__abs__ called...


16.278820596099706

<hr>

__Example 3__:

Arithmetic operators are not restricted to numbers.

Create a `Family` class which allows to add members.

In [26]:
class Person:
    def __init__(self, name):
        self.name = name
        
    def __repr__(self):
        return f"Person('{self.name}')"

In [27]:
class Family:
    def __init__(self, *member):
        self.members = []
        if len(member) > 0:
            for m in member:
                self.members.append(m)
        
    def __iadd__(self, other):
        self.members.append(other)
        return self
    

In [28]:
marshes = Family(Person('Randy'), Person('Sharon'))
id(marshes)

1659417699712

In [29]:
marshes.members

[Person('Randy'), Person('Sharon')]

In [30]:
marshes += Person('Shelly')
marshes += Person('Stan')

In [31]:
marshes.members

[Person('Randy'), Person('Sharon'), Person('Shelly'), Person('Stan')]

In [32]:
id(marshes)

1659417699712

<hr>

<a id='rich-comparisons'></a>
## 4. Rich Comparisons

`__lt__` < `__gt__`<br>
`__le__` <= `__ge__`<br>
`__eq__` == `__ne__`<br>
`__ne__` != `__eq__`<br>
`__gt__` > `__lt__`<br>
`__ge__` >= `__le__`

If `return NotImplemented` Python tries the reflection, e.g.:
* if `a < b` returns `NotImplemented` Python will try `b > a`


<hr>

__Example 4__:

If class don't provide `==`, Python will use `is`.

In [33]:
class Vector:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __repr__(self):
        return f'Vector(x={self.x}, y={self.y})'

In [34]:
v1 = Vector(0, 0)
v2 = Vector(0, 0)
print(id(v1), id(v2))

1659417748960 1659417687328


In [35]:
v1 == v2

False

In [36]:
class Vector:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __repr__(self):
        return f'Vector(x={self.x}, y={self.y})'
        
    def __eq__(self, other):
        print('__eq__ called...')
        if isinstance(other, tuple):
            ''' To compare with a tuple '''
            other = Vector(*other)
        if isinstance(other, Vector):
            return self.x == other.x and self.y == other.y
        return NotImplemented
    
    def __abs__(self):
        return sqrt(self.x ** 2 + self.y ** 2)
    
    def __lt__(self, other):
        print('__lt__ called...')
        if isinstance(other, tuple):
            other = Vector(*other)
        if isinstance(other, Vector):
            return abs(self) < abs(other)
        return NotImplemented
    
    def __le__(self, other):
        return self == other or self < other

<hr>

<a id='eq&ne'></a>
__Equal__ & __not equal__:

In [37]:
v1 = Vector(10, 11)
v2 = Vector(10, 11)
v3 = Vector(1, 1)

In [38]:
v1 == (10, 11)

__eq__ called...


True

In [39]:
v1 == v2, v1 == v3

__eq__ called...
__eq__ called...


(True, False)

If `a != b` (`a.__ne__(b)`) is not available, but Python uses `not(a == b)` instead

In [40]:
v1 != v2

__eq__ called...


False

<hr>

<a id='lt&gt'></a>
__Lower than__ & __greater than__:

In [41]:
v1 = Vector(0, 0)
v2 = Vector(1, 1)

In [42]:
v1 < v2

__lt__ called...


True

If `a > b` (`a.__gt__(b)`) is not available, Python will try reverse comparison `b < a`

In [43]:
v1 > v2

__lt__ called...


False

In [44]:
v1 <= v2

__eq__ called...
__lt__ called...


True

In [45]:
v1 >= v2

__eq__ called...
__lt__ called...


False

<hr>

__Example 5__:

`@total_ordering` decorator in the `functools` module, that will work with `__eq__` and **one** other rich comparison method

In [46]:
from functools import total_ordering

@total_ordering
class Number:
    def __init__(self, x):
        self.x = x
        
    def __eq__(self, other):
        print('__eq__ called...')
        if isinstance(other, Number):
            return self.x == other.x
        return NotImplemented
    
    def __lt__(self, other):
        print('__lt__ called...')
        if isinstance(other, Number):
            return self.x < other.x
        return NotImplemented

In [47]:
a = Number(1)
b = Number(2)
c = Number(1)

In [48]:
a < b

__lt__ called...


True

In [49]:
a <= b

__lt__ called...


True

In [50]:
a <= c

__lt__ called...
__eq__ called...


True

<hr>

<a id='hashing-and-equality'></a>
## 5. Hashing and Equality

By default, when we create a custom class, we inherit `__eq__` and `__hash__` from the object class.<br>
This means that by default custom classes produce hashable objects that can be used in mapping types such as dictionaries and sets.

* `__hash__` uses `id` of object
* `__eq__` uses identity comparison `is`

If we override the `__eq__` method, Python will automatically make our class `unhashable` and sets the `__hash__` property to `None`.

We can no longer use instances of this class as keys in a dictionary or elements of a set.

We can however provide our own override for `__hash__`.<br>
For this to work well in data structurfes such as dictionaries, what we use to create a hash of the class should remain immutable.

<hr>

__Example 6__:

In [51]:
class Person:
    def __init__(self, name):
        self._name = name
        
    @property
    def name(self):
        return self._name
    
    def __eq__(self, other):
        return isinstance(other, Person) and self.name == other.name
            
    def __hash__(self):
        return hash(self.name)

In [52]:
p1 = Person('John')
p2 = Person('John')
p3 = Person('Eric')

In [53]:
p1 == p2, p1 == p3

(True, False)

In [54]:
d = {p1: 'Eric'}

In [55]:
d

{<__main__.Person at 0x1825d04e160>: 'Eric'}

In [56]:
hash(p1)

-313983210192495358

<hr>

<a id='booleans'></a>
## 6. Booleans

The way Python determines the truth value of our custom classes is to:

1. first look for an implementation of the `__bool__` method (which needs to return a boolean)
2. if not present, looks for `__len__` and will return `False` if that is `0`, and `True` otherwise
3. otherwise returns `True`

<hr>

__Example 7__:

Custom class returns `True` if `__bool__` and `__len__` not specified

In [57]:
class Person:
    pass

In [58]:
p = Person()

In [59]:
bool(p)

True

<hr>

__Example 8__:

Boolean truth depending on `__len__` result

In [60]:
class MyList:
    def __init__(self, length):
        self._length = length
        
    def __len__(self):
        print('__len__ called')
        return self._length

In [61]:
l1 = MyList(0)  # so __len__ will return 0
l2 = MyList(10)  # so __len__ will return 10

In [62]:
bool(l1)

__len__ called


False

In [63]:
bool(l2)

__len__ called


True

<hr>

__Example 9__:

Consider a 2D `Point` class where we want to consider the origin point `(0,0)` falsy, and everything else truthy.

In [64]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __bool__(self):
        return bool(self.x or self.y)

In [65]:
p1 = Point(0, 0)
p2 = Point(1, 1)

In [66]:
bool(p1)

False

In [67]:
bool(p2)

True

<hr>

<a id='callables'></a>
## 7. Callables

__Example 10__:

Class instance is callable but instance will not.

In [68]:
class Person:
    def __init__(self, name):
        self.name = name

In [69]:
callable(Person)

True

In [70]:
p = Person('Eric')

In [71]:
callable(p)

False

__Example 11__: Cache with cache-miss counter.

Implement a dictionary to act as a cache, but also keep track of the cache missed.

The `defaultdict` class can be used as a cache with specified `callable` to use when requesting a non-existent key from `defaultdict`. <br>
Once called it creates a new key with default value.

In [72]:
from collections import defaultdict

class DefaultValue:
    def __init__(self, default_value):
        self.default_value = default_value
        self.counter = 0
        
    def __iadd__(self, other):
        if isinstance(other, int):
            self.counter += other
            return self
        raise ValueError('Can only increment with an integer value.')
        
    def __call__(self):
        self.counter += 1
        return self.default_value

In [73]:
cache_default_1 = DefaultValue(None)
cache_default_2 = DefaultValue(0)

cache_1 = defaultdict(cache_default_1)
cache_2 = defaultdict(cache_default_2)

<hr>

In [74]:
cache_default_1.counter, cache_default_2.counter

(0, 0)

In [75]:
cache_1['a'], cache_1['b'], cache_1['a']

(None, None, None)

In [76]:
cache_default_1.counter

2

<hr>

In [77]:
cache_2['a'], cache_2['b'], cache_2['c']

(0, 0, 0)

In [78]:
cache_default_2.counter

3

In [79]:
cache_2['e'] = 10

In [80]:
cache_2

defaultdict(<__main__.DefaultValue at 0x1825d0489a0>,
            {'a': 0, 'b': 0, 'c': 0, 'e': 10})

In [81]:
cache_default_2.counter

3

__Example 12:__ Profiling functions

Keep track of how many times functions are called and how long they took to run on average.

This will be done through function decorator, defined as a callable custom class.

In [82]:
from time import perf_counter

from time import sleep
import random

class Profiler:
    def __init__(self, fn):
        self.counter = 0
        self.total_elapsed = 0
        self.fn = fn
        
    def __call__(self, *args, **kwargs):
        self.counter += 1
        start = perf_counter()
        result = self.fn(*args, **kwargs)
        end = perf_counter()
        self.total_elapsed += (end - start)
        return result
        
    @property
    def avg_time(self):
        return self.total_elapsed / self.counter

<hr>

In [83]:
@Profiler
def func_1(a, b):
    sleep(random.random())
    return (a, b)

In [84]:
func_1(1, 2)

(1, 2)

In [85]:
func_1.counter

1

In [86]:
func_1(2, 3)

(2, 3)

In [87]:
func_1.counter

2

In [88]:
func_1(0, 4)

(0, 4)

In [89]:
func_1.counter

3

In [90]:
func_1.avg_time

0.5679931333333336

<hr>

In [91]:
@Profiler
def func_2():
    sleep(random.random())

In [92]:
func_2(), func_2(), func_2()

(None, None, None)

In [93]:
func_2.counter

3

In [94]:
func_2.avg_time

0.8270815000000002

<hr>

<a id='delete-method'></a>
## 8. Delete method

The garbage collector destroys objects that are no longer referenced anywhere.

The `__del__` method will get called right before the object is destroyed by the garbage collector.

There are some issues, hence it is better to avoid using `__del__` and use `context managers` instead. 

<hr>

<a id='format-method'></a>
## 9. Format method

`format()` function can be used to precisely format certain types, such as floats or datetime.

In [95]:
a = 0.1
format(a, '.20f')

'0.10000000000000000555'

In [96]:
from datetime import datetime

now = datetime.utcnow()

now

datetime.datetime(2021, 6, 30, 20, 33, 14, 282294)

In [97]:
format(now, '%a %Y-%m-%d  %I:%M %p')

'Wed 2021-06-30  08:33 PM'

<hr>

This can be included into a custom class by defining `__format__` method.

In [98]:
class Person:
    def __init__(self, name, dob):
        self.name = name
        self.dob = dob
        
    def __repr__(self):
        print('__repr__ called...')
        return f'Person(name={self.name}, dob={self.dob.isoformat()})'
    
    def __str__(self):
        print('__str__ called...')
        return f'Person({self.name})'
    
    def __format__(self, date_format_spec):
        print(f'__format__ called with {repr(date_format_spec)}...')
        dob = format(self.dob, date_format_spec)
        return f'Person(name={self.name}, dob={dob})'

In [99]:
from datetime import date

p = Person('Alex', date(1900, 10, 20))

In [100]:
str(p)

__str__ called...


'Person(Alex)'

In [101]:
repr(p)

__repr__ called...


'Person(name=Alex, dob=1900-10-20)'

In [102]:
format(p, '%B %d, %Y')

__format__ called with '%B %d, %Y'...


'Person(name=Alex, dob=October 20, 1900)'

If we do not specify a format, then the `format` function will use an empty string:

In [103]:
format(p)

__format__ called with ''...


'Person(name=Alex, dob=1900-10-20)'